# Train Models Using LeRobot on MI300x

This guide walks you through setting up environment for training imitation learning policies using LeRobot library on a DigitalOcean (DO) instance equipped with AMD MI300x GPUs and ROCm.

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=${HF_USER}/${DATASET_NAME}`).
  If you don’t have an access token yet, you can sign up for Hugging Face [here](https://huggingface.co/join). After signing up, create an access token by visiting [here](https://huggingface.co/settings/tokens).
- A wandb account to enable training visualization and upload your training evidence to our github.
  You can sign up for Wandb [here](https://wandb.ai/signup) and visit [here](https://wandb.ai/authorize) to create a token.
- Access to DO instance AMD Mi300x GPU


## Verify ROCm and GPU availability
This cell uses `pytorch` to check AMD GPU Info. The expected ouput is 
```
CUDA compatible device availability: True
device name [0]: AMD Instinct MI300X VF
```

In [1]:
import torch
print(f'CUDA compatible device availability:',torch.cuda.is_available())
print(f'device name [0]:', torch.cuda.get_device_name(0))


CUDA compatible device availability: True
device name [0]: AMD Instinct MI300X VF


## Install FFmpeg 7.x
This cell uses `apt` to install ffmpeg 7.x for LeRobot.

In [2]:
!add-apt-repository ppa:ubuntuhandbook1/ffmpeg7 -y # install PPA which contains ffmpeg 7.x
!apt update && apt install ffmpeg -y

Repository: 'Types: deb
URIs: https://ppa.launchpadcontent.net/ubuntuhandbook1/ffmpeg7/ubuntu/
Suites: noble
Components: main
'
Description:
unofficial build for FFmpeg 7 for Ubuntu 22.04 | 24.04, backport from Debian's deb.multimedia.org repository

If the packages here are helpful, you may buy me a coffee:

         https://ko-fi.com/ubuntuhandbook1
More info: https://launchpad.net/~ubuntuhandbook1/+archive/ubuntu/ffmpeg7
Adding repository.
Hit:1 https://repo.radeon.com/amdgpu/30.10/ubuntu jammy InRelease
Hit:2 https://repo.radeon.com/rocm/apt/7.0 jammy InRelease                     
Hit:3 https://repo.radeon.com/graphics/7.0/ubuntu jammy InRelease              
Hit:4 http://archive.ubuntu.com/ubuntu noble InRelease                         
Get:5 http://archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]        
Get:6 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]      
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu noble InRelease [17.8 kB]


## Install LeRobot v0.4.1
This cell clones the `lerobot` repository from Hugging Face, and installs the package in editable mode. Extra Features: To install additional dependencies for training SmolVLA or Pi models, refer to the [LeRobot offical page](https://huggingface.co/docs/lerobot/index). 


In [3]:
!git clone https://github.com/huggingface/lerobot.git
!cd lerobot && git checkout -b v0.4.1 v0.4.1 # let’s synchronize using this version
!cd lerobot && pip install -e .

Cloning into 'lerobot'...
remote: Enumerating objects: 45396, done.
remote: Counting objects: 100% (405/405), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 45396 (delta 331), reused 217 (delta 217), pack-reused 44991 (from 4)
Receiving objects: 100% (45396/45396), 232.01 MiB | 76.49 MiB/s, done.
Resolving deltas: 100% (29278/29278), done.
Switched to a new branch 'v0.4.1'
Obtaining file:///workspace/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 95.6 MB/s  0:00:006m0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 115.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
pip install -e ".[smolvla]"

Obtaining file:///workspace

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
ERROR: file:///workspace does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
Note: you may need to restart the kernel to use updated packages.


## Weights & Biases login
This cell install and log into Weights & Biases (wandb) to enable experiment tracking and logging.

In [ ]:
!pip install wandb
import wandb
wandb.login(key="")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


/opt/venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/opt/venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` st

True

## Login into Hugging Face Hub

In [ ]:
from huggingface_hub import login
login(token="")

## Start Training Models with LeRobot

This cell uses the lerobot-train CLI from the lerobot library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=YOUR_HF_USERNAME/YOUR_DATASET`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `lerobot/svla_so100_pickplace`.

2. `--policy.type=act`:  
   Specifies the policy configuration to use. `act` refers to [configuration_act.py](../lerobot/common/policies/act/configuration_act.py), which will automatically adapt to your dataset’s setup (e.g., number of motors and cameras).

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.The name typically includes the model type (e.g., act, smolvla), the dataset name, and additional descriptive tags.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an AMD or NVIDIA GPU. 

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

7. `--policy.push_to_hub=`:
   To automatically upload the trained policy to the Hugging Face Hub, you must specify `--policy.repo_id` (e.g., ${HF_USER}/{REPO_NAME}).

In [20]:
from huggingface_hub import HfApi

hub_api = HfApi()
hub_api.delete_tag("NotARoomba/synapse_3", tag="3.0", repo_type="dataset")

In [37]:
!lerobot-train \
  --policy.path=lerobot/smolvla_base \
  --policy.repo_id=NotARoomba/synapse_smvla_5 \
  --dataset.repo_id=NotARoomba/synapse_5 \
  --rename_map='{"observation.images.side": "observation.images.camera1", "observation.images.top": "observation.images.camera2", "observation.images.wrist": "observation.images.camera3"}' \
  --batch_size=20 \
  --steps=3000 \
  --output_dir=outputs/train/synapse_smvla_5 \
  --job_name=synapse_smvla_5 \
  --policy.device=cuda \
  --policy.push_to_hub=true \
  --wandb.enable=true

INFO 2025-12-21 19:11:48 ot_train.py:163 {'batch_size': 20,
 'checkpoint_path': None,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'affine': {'kwargs': {'degrees': [-5.0,
                                                                            5.0],
                                                                'translate': [0.05,
                                                                              0.05]},
                                                     'type': 'RandomAffine',
                                                     'weight': 1.0},
                                          'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         '

In [48]:
!huggingface-cli upload NotARoomba/eval_synapse_smvla_5_v2 outputs/train/eval_synapse_smvla_5_v2/checkpoints/last/pretrained_model

⚠️  Warning: 'huggingface-cli upload' is deprecated. Use 'hf upload' instead.
Start hashing 7 files.
Finished hashing 7 files.
Processing Files (0 / 0)      : |                  |  0.00B /  0.00B            
New Data Upload               : |                  |  0.00B /  0.00B            

  ...zer_processor.safetensors: 100%|██████████████| 8.51kB / 8.51kB            


  ...zer_processor.safetensors: 100%|██████████████| 8.51kB / 8.51kB            

  ...zer_processor.safetensors: 100%|██████████████| 8.51kB / 8.51kB            


Processing Files (2 / 2)      :   0%|              | 17.0kB / 1.20GB,   ???B/s  

  ...zer_processor.safetensors: 100%|██████████████| 8.51kB / 8.51kB            


  ...zer_processor.safetensors: 100%|██████████████| 8.51kB / 8.51kB            



  ...d_model/model.safetensors:   5%|▋             | 58.7MB / 1.20GB            

  ...zer_processor.safetensors: 100%|██████████████| 8.51kB / 8.51kB            


  ...zer_processor.safetensors: 100%|███████████

In [46]:
!lerobot-train \
  --policy.pretrained_path=NotARoomba/eval_synapse_smvla_5 \
  --policy.type=smolvla \
  --policy.repo_id=NotARoomba/eval_synapse_smvla_5_v2 \
  --dataset.repo_id=NotARoomba/synapse_5 \
  --save_checkpoint=true \
  --save_freq=500 \
  --batch_size=20 \
  --steps=3000 \
  --output_dir=outputs/train/eval_synapse_smvla_5_v2 \
  --job_name=synapse_smvla_5_v2 \
  --policy.device=cuda \
  --policy.push_to_hub=true \
  --wandb.enable=true 

Traceback (most recent call last):
  File "/opt/venv/bin/lerobot-train", line 7, in <module>
    sys.exit(main())
             ^^^^^^
  File "/workspace/lerobot/src/lerobot/scripts/lerobot_train.py", line 444, in main
    train()
  File "/workspace/lerobot/src/lerobot/configs/parser.py", line 233, in wrapper_inner
    response = fn(cfg, *args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspace/lerobot/src/lerobot/scripts/lerobot_train.py", line 143, in train
    cfg.validate()
  File "/workspace/lerobot/src/lerobot/configs/train.py", line 110, in validate
    raise FileExistsError(
FileExistsError: Output directory outputs/train/eval_synapse_smvla_5_v2 already exists and resume is False. Please change your output directory so that outputs/train/eval_synapse_smvla_5_v2 is not overwritten.


In [30]:
!lerobot-train \
  --dataset.repo_id=${HF_USER}/record-test \
  --batch_size=64 \
  --steps=15000 \
  --output_dir=outputs/train/act_so101_cube_15ksteps \
  --job_name=act_so101_3cube_1ksteps \
  --policy.repo_id=${HF_USER}/act_so101_cube_15ksteps \
  --policy.device=cuda \
  --policy.type=act \
  --policy.push_to_hub=true \
  --wandb.enable=true

INFO 2025-12-21 18:03:58 ot_train.py:163 {'batch_size': 64,
 'checkpoint_path': None,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'affine': {'kwargs': {'degrees': [-5.0,
                                                                            5.0],
                                                                'translate': [0.05,
                                                                              0.05]},
                                                     'type': 'RandomAffine',
                                                     'weight': 1.0},
                                          'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         '

**Notes**:

- If using a local dataset, add `--dataset.root=/path/to/dataset`.
- Adjust `--batch_size` and `--steps` based on your hardware and dataset.
- Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`
- Training progress visualized in your wandb dashboard


## Download Models from Hugging Face to Local Machine
Now after training is done, you can download the model to local machine. 

In [ ]:
!huggingface-cli download ${HF_USER}/{REPO_NAME} --repo-type model --local-dir path/to/model
# e.g. huggingface-cli upload ${HF_USER}/act_so101_3cube_1ksteps \
#  outputs/train/act_so101_3cube_1ksteps/checkpoints/last/pretrained_model

## Miscs
1. Once the environment is setup, you can open a terminal session for training by navigating to `File → New Launcher → Other → Terminal`.
2. You can also upload your datasets to the container by clicking the `Upload Files` button in the left pane.

## Q&A
1. If you encounter an error like:
   ```
   FileExistsError: Output directory outputs/train/act_so101_3cube_1ksteps already exists and resume is False. Please change your output directory so that outputs/train/act_so101_3cube_1ksteps is not overwritten. 
   ```
   Remove the existing directory before proceeding:

In [ ]:
!rm -fr outputs/train/act_so101_3cube_1ksteps

2. When running models other than ACT, ensure you install the required additional dependencies for those models.

In [ ]:
# For smolVLA
!cd lerobot && pip install -e ".[smolvla]"
# For Pi
!cd lerobot && pip install -e ".[pi]"

3. If you want to resume the training from last checkpoint, run the command below:

In [ ]:
!lerobot-train \
  --resume=true \
  --config_path=outputs/train/<job name>/checkpoints/last/pretrained_model/train_config.json \
  --steps=<new total steps>

4. If you want to upload your dataset using `huggingface-cli upload <repo name> <path to the dataset> --repo-type=dataset`, be sure to set a codebase tag like below:

In [ ]:
from huggingface_hub import HfApi
from huggingface_hub import login

login(token="your_huggingface_token")
hub_api = HfApi()
hub_api.create_tag(<HF_REPO_NAME>, tag="v3.0", revision="main", repo_type="dataset")